In [569]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter, defaultdict

In [554]:
np.random.seed(12)
num_observations, num_features = 200, 100

x1 = np.random.randint(low =0, high = 3, size=(num_observations, num_features))
y = np.random.randint(low =0, high = 2, size=(num_observations, 1))

In [555]:
# calculate prior 
prior0, prior1 = len(y[np.where(y==0)])/len(y), len(y[np.where(y==1)])/len(y)

In [556]:
# comfirm the prior 
x1.shape, y.shape, prior0*num_observations, prior1*num_observations, x1[np.where(y==0), :].shape, x1[np.where(y==1), :].shape

((200, 100), (200, 1), 97.0, 103.0, (2, 97, 100), (2, 103, 100))

In [570]:
## calculate likelihood
likelihood = defaultdict(dict)

for c in np.unique(y):
    for f in range(x1.shape[1]):
        subset = x1[np.where(y==c)[0]][:,f]
        labels = np.unique(x1[:,f])
        for l in labels:
            likelihood[(c, f, l)] = np.apply_along_axis(lambda x, y: np.count_nonzero(x==l)/y, 0, subset, subset.shape[0])

In [571]:
len(likelihood.keys())

600

In [572]:
## calculate the posterior prob
res = defaultdict(list)

for c in np.unique(y):       
    for idx in range(x1.shape[0]):
        if c == 1:
            prod_ = prior1
            evidence = prior0
            e = 0
        else:
            prod_ = prior0
            evidence = prior1
            e = 1
        item= x1[idx]
        for col in range(len(item)):
            prod_ *= likelihood[(c, col, item[col])]
            evidence *= likelihood[(e, col, item[col])]
        
        res[idx].append(prod_/ (evidence + prod_))


In [573]:
# confirmation
from sklearn.naive_bayes import CategoricalNB
clf = CategoricalNB()
clf.fit(x1, y.reshape(y.shape[0],))
clf_ans = clf.predict(x1)

In [574]:
# range of original calculation
(np.min(clf.predict_proba(x1)), np.max(clf.predict_proba(x1)))

(0.00039072554417151343, 0.9996092744558326)

In [575]:
# get the final category
calculated_ans = np.argmax(np.array(list(res.values())),axis=1)

In [576]:
calculated_ans == clf_ans

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [577]:
# comparing the probability difference 
diff_= clf.predict_proba(x1)- np.array(list(res.values()))

In [578]:
# difference range
np.min(diff_), np.max(diff_)

(-0.008623979106333834, 0.00862397910632648)

In [582]:
## calculate the posterior log prob
log_res = defaultdict(list)
for c in np.unique(y):       
    for idx in range(x1.shape[0]):
        if c == 1:
            prod_ = np.log(prior1)
            evidence = np.log(prior0)
            e = 0
        else:
            prod_ = np.log(prior0)
            evidence = np.log(prior1)
            e = 1
            
        item= x1[idx]
        for col in range(len(item)):
            prod_ += np.log(likelihood[(c, col, item[col])])
            #evidence += np.log(likelihood[(e, col, item[col])])
               
        log_res[idx].append(prod_)

In [583]:
calculated_log_ans = np.argmax(np.array(list(log_res.values())),axis=1)

In [584]:
calculated_log_ans == clf_ans

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,